# Lesson 4: Form Parsing

**Lesson objective**: Incorporate form parsing to the workflow

In your previous lesson, you used LlamaParse to parse a resume, and included parsing instructions. You'll do that again this time, but the instructions are going to be more advanced -- you're going to get it to read an application form and convert it into a list of fields that need to be filled in, and return that as a JSON object. You will then incorporate these steps in the workflow you started building in the previous lesson.

<div style="background-color:#fff1d7; padding:15px;"> <b> Note</b>: Make sure to run the notebook cell by cell. Please try to avoid running all cells at once.</div>

In [1]:
import getpass
import os

INFERENCE_SERVER_URL = "http://localhost:8989"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"

API_KEY= "alanliuxiang"

from llama_index.llms.openai_like import OpenAILike

model = OpenAILike(
  model=MODEL_NAME,
  api_key=API_KEY,
  api_base= f"{INFERENCE_SERVER_URL}/v1",
  context_window=1234,
  is_chat_model=True,  # supports chat completions
  is_function_calling_model=True # supports tools/functions in the api
)


In [2]:
import os, json
# from llama_parse import LlamaParse
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)
from helper import get_openai_api_key, get_llama_cloud_api_key
from IPython.display import display, HTML
from helper import extract_html_content
from llama_index.utils.workflow import draw_all_possible_flows

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Parsing an Application Form with LlamaParse

Let's create a parser with our new parsing instructions, including formatting instructions.

In [4]:
# parser = LlamaParse(
#     api_key=llama_cloud_api_key,
#     base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
#     result_type="markdown",
#     content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
#     formatting_instruction="Return a bulleted list of the fields ONLY."
# )


# load pdf
# from langchain_community.document_loaders import PyPDFLoader
# loader_A = PyPDFLoader("./data/fake_application_form.pdf")
# documents = loader_A.load()

from llama_index.core import SimpleDirectoryReader
# documents = SimpleDirectoryReader("./data/fake_application_form.pdf").load_data()

reader = SimpleDirectoryReader(
    input_files=["./data/fake_application_form.pdf"]
)

documents = reader.load_data()

In [5]:
# result = parser.load_data("data/fake_application_form.pdf")[0]

result = documents[0]

As print out the results, you can see it has pulled all the boxes in the form and correctly turned them into field names to be filled in.

In [6]:
# print(result.text)

A useful thing LLMs can do is turn human-readable formats into machine-readable ones. You will ask the LLM to turn the list into a JSON object with a list of fields.

In [7]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embed_model = HuggingFaceEmbedding()

Settings.llm = model
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 4096

In [8]:
# llm = OpenAI(model="gpt-4o-mini")

llm=model

In [9]:
raw_json = llm.complete(
    f"""
    This is a parsed form.
    Convert it into a JSON object containing only the list 
    of fields to be filled in, in the form {{ fields: [...] }}. 
    <form>{result.text}</form>. 
    Return JSON ONLY, no markdown."""
)

Here's the raw JSON:

In [10]:
raw_json.text

'{\n  "fields": [\n    "First Name",\n    "Last Name",\n    "Email",\n    "Phone",\n    "Linkedin",\n    "Project Portfolio",\n    "Degree",\n    "Graduation Date",\n    "Current Job Title",\n    "Current Employer",\n    "Technical Skills",\n    "Describe why you’re a good fit for this position",\n    "Do you have 5 years of experience in React?"\n  ]\n}'

And you can print out the fields one by one.

In [11]:
fields = json.loads(raw_json.text)["fields"]

for field in fields:
    print(field)

First Name
Last Name
Email
Phone
Linkedin
Project Portfolio
Degree
Graduation Date
Current Job Title
Current Employer
Technical Skills
Describe why you’re a good fit for this position
Do you have 5 years of experience in React?


Now that you know how to parse the job application form, you will add this processing to the workflow of lesson 3. You will do that in two steps as shown in this figure:

<img width="700" src="images/L4-diagrams.png">

## Adding a Form Parser to the Workflow (first update)

Now let's take the workflow you built in the last lesson and add your parser.

In [12]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str

Your `set_up` step now emits a `ParseFormEvent` which triggers your new step, `parse_form`. For the moment you can leave the ask_questions step untouched, it will be updated in another section of this notebook.

In [13]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: model
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        # self.llm = OpenAI(model="gpt-4o-mini")
        self.llm = model

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            # documents = LlamaParse(
            #     api_key=llama_cloud_api_key,
            #     base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            #     result_type="markdown",
            #     content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            # ).load_data(ev.resume_file)
            documents = SimpleDirectoryReader(ev.resume_file).load_data()
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=embed_model
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in, 
        # you'll be generating the queries instead 
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        # parser = LlamaParse(
        #     api_key=llama_cloud_api_key,
        #     base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
        #     result_type="markdown",
        #     content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
        #     formatting_instruction="Return a bulleted list of the fields ONLY."
        # )

        # get the LLM to convert the parsed form into JSON
        # result = parser.load_data(ev.application_form)[0]

        # result = SimpleDirectoryReader(ev.application_form).load_data()

        reader = SimpleDirectoryReader(
            input_files=[ev.application_form]
        )
        
        result = reader.load_data()
        
        raw_json = self.llm.complete(
            f"""
            This is a parsed form. 
            Convert it into a JSON object containing only the list 
            of fields to be filled in, in the form {{ fields: [...] }}. 
            <form>{result}</form>. 
            Return JSON ONLY, no markdown.
            """)
        
        fields = json.loads(raw_json.text)["fields"]

        for field in fields:
            print(field)
        return StopEvent(result="Dummy event")

    # will be edited in the next section
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return StopEvent(result=response.response)

In [14]:
w = RAGWorkflow(timeout=60, verbose=False)
result = await w.run(
    resume_file="./data/fake_resume.pdf",
    application_form="./data/fake_application_form.pdf"
)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.
id_
embedding
metadata
relationships
text_resource
path
url
mimetype


## Generating Questions (second update)

Excellent! Your workflow knows what fields it needs answers for. In this next iteration, you can fire off one `QueryEvent` for each of the fields, so they'll be executed concurrently (we talked about doing concurrent steps in Lesson 2). 

<img width="700" src="images/L4-diag-2.png">


The changes you're going to make are:
* Generate a `QueryEvent` for each of the questions you pulled out of the form
* Create a `fill_in_application` step which will take all the responses to the questions and aggregate them into a coherent response
* Add a `ResponseEvent` to pass the results of queries to `fill_in_application`

In [15]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str
    field: str

# new!
class ResponseEvent(Event):
    response: str

In [16]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: model
    query_engine: VectorStoreIndex


    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        # self.llm = OpenAI(model="gpt-4o-mini")
        self.llm = model

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            documents = SimpleDirectoryReader(ev.resume_file).load_data()
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=embed_model
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in, 
        # you'll be generating the queries instead 
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        reader = SimpleDirectoryReader(
            input_files=[ev.application_form]
        )
        
        result = reader.load_data()
        raw_json = self.llm.complete(
            f"""
            This is a parsed form. 
            Convert it into a JSON object containing only the list 
            of fields to be filled in, in the form {{ fields: [...] }}. 
            <form>{result}</form>. 
            Return JSON ONLY, no markdown.
            """)
        fields = json.loads(raw_json.text)["fields"]

        # new!
        # generate one query for each of the fields, and fire them off
        for field in fields:
            ctx.send_event(QueryEvent(
                field=field,
                query=f"How would you answer this question about the candidate? {field}"
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)

    # new!
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)
        return StopEvent(result=result)

In [17]:
w = RAGWorkflow(timeout=120, verbose=False)
result = await w.run(
    resume_file="./data/fake_resume.pdf",
    application_form="./data/fake_application_form.pdf"
)
print(result)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.


/tmp/ipykernel_5855/2296005844.py:88: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("total_fields", len(fields))
/tmp/ipykernel_5855/2296005844.py:100: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  total_fields = await ctx.get("total_fields")


Field: First Name
Response: Sarah

Field: Last Name
Response: Chen

Field: Email
Response: sarah.chen@email.com

Field: Phone
Response: No direct information available.

Field: Linkedin
Response: LinkedIn profile can be accessed at linkedin.com/in/sarahchen.

Field: Project Portfolio
Response: EcoTrack (full-stack application with React, Node.js, MongoDB, and machine learning for sustainability recommendations) and ChatFlow (real-time chat application using WebSocket protocol and React with end-to-end encryption and message persistence).

Field: Degree
Response: Bachelor of Science in Computer Science from the University of California, Berkeley, graduated in 2017.

Field: Graduation Date
Response: Graduated in 2017.

Field: Current Job title
Response: Senior Full Stack Developer

Field: Current Employer
Response: TechFlow Solutions, San Francisco, CA

Field: Technical Skills
Response: Frontend: React.js, Redux, Next.js, TypeScript, Vue.js, Nuxt.js, HTML5, CSS3, SASS/SCSS, Jest, React T

## Workflow Visualization

You can visualize the workflow you just created.

In [18]:
WORKFLOW_FILE = "workflows/form_parsing_workflow.html"
draw_all_possible_flows(w, filename=WORKFLOW_FILE)
html_content = extract_html_content(WORKFLOW_FILE)
display(HTML(html_content), metadata=dict(isolated=True))

workflows/form_parsing_workflow.html


## Cool!

Your workflow takes all the fields in the form and generates plausible answers for all of them. There are a couple of fields where I think it can do better, and in the next lesson you'll add the ability to give that feedback to the agent and get it to try again.